In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
import json
import crypto.get_dca
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
path = 'C:\\Users\\l.mcleary\\OneDrive - Accenture\\Documents\\Python Scripts\\crypto\\data\\'
file_path = path + max(os.listdir(path))
ordersDF = spark.read.json(file_path)

In [ ]:
ordersDF.show(n=1, vertical=True)

In [ ]:
ordersDF.select('status').distinct().show()

In [ ]:
ordersfilledDF = ordersDF.filter(ordersDF.status!='CANCELED')

In [ ]:
ordersfilledDF = ordersfilledDF.withColumn(
    "target_pair",
    F.when(F.col("target_pair")==F.col("symbol"),
    F.col("symbol")
    ).otherwise(F.col("target_pair")))

In [ ]:
buyDF = ordersfilledDF.filter(ordersfilledDF.side=="BUY").groupby('target_pair').agg({"executedQty":"sum"})
sellDF = ordersfilledDF.filter(ordersfilledDF.side=="SELL").groupby('target_pair').agg({"executedQty":"sum"})

In [ ]:
buyDF = buyDF.withColumnRenamed("sum(executedQty)", "Buy Total")
sellDF = sellDF.withColumnRenamed("sum(executedQty)", "Sell Total")

In [ ]:
sellDF.show()

In [ ]:
df = buyDF.join(sellDF, on="target_pair",how="FULL")

for col_name in df.columns:
    df = df.fillna(0)
df = df.withColumn("sum", F.col("Buy Total") - F.col("Sell Total"))

In [ ]:
df.show()

In [ ]:
coins = df.select("target_pair").collect()

In [ ]:
from crypto.get_dca import client

In [ ]:
from binance.spot import Spot

client = Spot()

In [ ]:
current_price = {}
for c in coins:
    print(c['target_pair'])
    try:
        current_price[c['target_pair']] = client.avg_price(c['target_pair'])['price']
    except:
        print('failed: ' +str(c) )

In [ ]:
current_price

In [ ]:
cp= list(map(list, {k: float(v) for k, v in current_price.items()}.items()))
cp


In [ ]:
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, FloatType, DecimalType
schema = StructType([
    StructField('Coin', StringType()),
    StructField('Price', FloatType())
    ])
currentDF = spark.createDataFrame(cp, schema)

In [ ]:
df = df.join(currentDF, F.col("target_pair")== F.col("Coin"), "Full")

In [ ]:
if df.filter("target_pair!=Coin").count()==0:
    df = df.drop("Coin")
df = df.drop("Buy Total", "Sell Total")
df = df.withColumn("total_value", F.col("sum")*F.col("price"))

In [ ]:
df = df.orderBy("total_value", ascending=False)
df.show()

In [ ]:
pair_value_list = df.select("target_pair", "total_value") \
    .filter(F.col("total_value").isNotNull()).collect()
target_pair = [p["target_pair"] for p in pair_value_list]
total_value = [p["total_value"] for p in pair_value_list]


In [ ]:
target_pair = target_pair[4:]
total_value = total_value[4:]

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(total_value, labels=target_pair, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()